In [43]:
import json
import pandas as pd
from fuzzywuzzy import process


In [15]:
# read in V-Dem data
Dem=pd.read_csv("refugee_data/V-Dem-CY-Core-v12.csv")


In [ ]:
columnList=["country_name","year","v2xeg_eqdr","v2x_libdem"]
country_dem=Dem[columnList]

In [ ]:
country_dem.head()

In [34]:
# read in country border data
country_border = open('refugee_data/country_border_data.json')
countries_that_border = json.load(country_border)

In [28]:
# read in historic gdp pop data
gdp_per_cap_historic=pd.read_csv("refugee_data/gdp_per_cap.csv")


In [29]:
gdp_per_cap_historic.head()

,Series Name,Series Code,Country Name,Country Code,1990 [YR1990],2000 [YR2000],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,GDP per capita (current US$),NY.GDP.PCAP.CD,Afghanistan,AFG,..,..,591.190030208565,638.845851607289,624.315454506615,614.223342360392,556.007220861443,512.012778129357,516.679862210543,485.668418729808,494.179349886029,516.747870807558
1,GDP per capita (current US$),NY.GDP.PCAP.CD,Albania,ALB,617.230435515505,1126.68334010717,4437.14261222684,4247.63004748194,4413.06200528903,4578.63320812155,3952.80253807527,4124.05538986272,4531.0193737689,5287.66369446913,5395.65953185375,5246.29230634793
2,GDP per capita (current US$),NY.GDP.PCAP.CD,Algeria,DZA,2408.82298523208,1765.02714611601,5455.67940349747,5592.22011465753,5499.58733105334,5493.05669453687,4177.8895415169,3946.45244712537,4109.69821695727,4142.01855842917,3989.66827645236,3306.85820838104
3,GDP per capita (current US$),NY.GDP.PCAP.CD,American Samoa,ASM,..,..,10223.2983588916,11496.5240977923,11450.7241954879,11525.1563872309,12059.6351646776,12038.2496994923,11003.8297642807,11521.6097798453,11715.3601388487,12844.9009909959
4,GDP per capita (current US$),NY.GDP.PCAP.CD,Andorra,AND,18878.8523130643,21854.2468030614,43334.8114127088,38686.4612635075,39540.7248137461,41303.9293714464,35770.7767044085,37475.6350587526,38964.904478265,41791.9698372411,40897.3308728669,..


In [33]:
historic_pop=pd.read_csv('refugee_data/historic_pop.csv')
historic_pop.head()

,Series Name,Series Code,Country Name,Country Code,2000 [YR2000],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,"Population, total",SP.POP.TOTL,Afghanistan,AFG,20779957,28394806,29185511,30117411,31161378,32269592,33370804,34413603,35383028,36296111,37171922,38041757,38928341
1,"Population, total",SP.POP.TOTL,Albania,ALB,3089027,2927519,2913021,2905195,2900401,2895092,2889104,2880703,2876101,2873457,2866376,2854191,2837743
2,"Population, total",SP.POP.TOTL,Algeria,DZA,31042238,35333882,35977451,36661438,37383899,38140135,38923688,39728020,40551398,41389174,42228415,43053054,43851043
3,"Population, total",SP.POP.TOTL,American Samoa,ASM,57816,56675,56084,55755,55669,55717,55791,55806,55739,55617,55461,55312,55197
4,"Population, total",SP.POP.TOTL,Andorra,AND,65390,84461,84454,83748,82427,80770,79213,77993,77295,76997,77008,77146,77265


## Run model on new conflict

In [321]:
# user will set these values.
country= "South Sudan"
conflict_start=2021-1

In [322]:
touching_list=countries_that_border[country]

In [323]:
touching_list

['Central African Republic',
 'Congo (the Democratic Republic of the)',
 'Ethiopia',
 'Kenya',
 'Sudan',
 'Uganda']

In [324]:
touching_df = pd.DataFrame (touching_list, columns = ['bording_countries'])

In [325]:
options=historic_pop["Country Name"]

In [326]:
touching_df['historic_pop']=None

In [327]:
#Get data from historic populations given the year.
historic_pop_cols=historic_pop.columns
indexed_col={}
for i,c in enumerate(historic_pop_cols):
    indexed_col[i]=c

column,ratio_year, year_column_idx = process.extractOne(str(conflict_start), indexed_col)


for kk ,border in touching_df.iterrows():
    country,ratio,ind =process.extractOne(border["bording_countries"], options)
    touching_df.loc[kk,"historic_pop"]=historic_pop.at[ind,column]
    

In [328]:
# collect gdp per cap

options=gdp_per_cap_historic["Country Name"]
touching_df['historic_gdp_per_cap']=None
#Get data from historic gdp per cap given the year.
historic_gdp_cols=gdp_per_cap_historic.columns
indexed_gdp_col={}
for i,c in enumerate(historic_gdp_cols):
    indexed_gdp_col[i]=c

    
column,ratio_year, year_column_idx = process.extractOne(str(conflict_start), indexed_col)

for kk ,border in touching_df.iterrows():
    country,ratio,ind =process.extractOne(border["bording_countries"], options)
    touching_df.loc[kk,"historic_gdp_per_cap"]=gdp_per_cap_historic.at[ind,column]

In [329]:
# calculate historic gdp
touching_df['historic_gdp']=None
for kk ,row in touching_df.iterrows():
    print(row['historic_gdp_per_cap'])
    try:
        touching_df.loc[kk,"historic_gdp"]=int(row['historic_pop'])*float(row['historic_gdp_per_cap'])
    except Exception as e:
        print(e, f'GDP per cap for {row["bording_countries"]} data was {row["historic_gdp_per_cap"]}' )


492.79587119399
25194.0251444381
936.340461064749
1878.58070251348
486.418816281925
822.027681531666


In [330]:
touching_df

,bording_countries,historic_pop,historic_gdp_per_cap,historic_gdp
0,Central African Republic,4829764,492.79587119399,2380087758.04137
1,Congo (the Democratic Republic of the),393248,25194.0251444381,9907499999.999994
2,Ethiopia,114963583,936.340461064749,107645054311.875534
3,Kenya,53771300,1878.58070251348,101013726529.06308
4,Sudan,43849269,486.418816281925,21329109521.807709
5,Uganda,45741000,822.027681531666,37600368180.939934


In [331]:
country_dem.head()

,country_name,year,v2xeg_eqdr,v2x_libdem
0,Mexico,1789,NaN,0.04
1,Mexico,1790,NaN,0.04
2,Mexico,1791,NaN,0.04
3,Mexico,1792,NaN,0.04
4,Mexico,1793,NaN,0.04


In [332]:
touching_df["v2x_libdem"]=None
touching_df["v2xeg_eqdr"]=None

options=country_dem['country_name'].unique()

for kk ,row in touching_df.iterrows():
        
    country,ratio =process.extractOne(row["bording_countries"], options)
    lib=country_dem.loc[(country_dem["country_name"]==country) & (country_dem["year"]==int(conflict_start))]['v2x_libdem']
    eqdr=country_dem.loc[(country_dem["country_name"]==country) & (country_dem["year"]==int(conflict_start))]['v2xeg_eqdr']
    touching_df.loc[kk,"v2xeg_eqdr"]=eqdr.to_list()[0]
    touching_df.loc[kk,"v2x_libdem"]=lib.to_list()[0]

In [333]:
touching_df

,bording_countries,historic_pop,historic_gdp_per_cap,historic_gdp,v2x_libdem,v2xeg_eqdr
0,Central African Republic,4829764,492.79587119399,2380087758.04137,0.218,0.131
1,Congo (the Democratic Republic of the),393248,25194.0251444381,9907499999.999994,0.157,0.221
2,Ethiopia,114963583,936.340461064749,107645054311.875534,0.161,0.527
3,Kenya,53771300,1878.58070251348,101013726529.06308,0.374,0.532
4,Sudan,43849269,486.418816281925,21329109521.807709,0.086,0.193
5,Uganda,45741000,822.027681531666,37600368180.939934,0.236,0.392
